In [1]:
import cv2
import mediapipe as mp

## Leitura dos pontos do corpo

In [2]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_mao = mp.solutions.hands
mao = mp_mao.Hands()
mp_draw = mp.solutions.drawing_utils

# Captura da imagem

In [ ]:
def processamento_pontos_corpo(frame, pontos_pose):
    h,w,_ = frame.shape
    pontos_corpo = []  

    mp_draw.draw_landmarks(frame, pontos_pose, mp_pose.POSE_CONNECTIONS)

    for id, cord in enumerate(pontos_pose.landmark):
        cx, cy = int(cord.x * w), int(cord.y * h)
        cv2.circle(frame, (cx,cy), 4, (255,0,0), -1)
        pontos_corpo.append((cx, cy))

    return pontos_corpo

In [4]:
def processamento_pontos_mao(frame,pontos_mao):
    h, w, _ = frame.shape
    pontos = []

    for points in pontos_mao:
        mp_draw.draw_landmarks(frame, points, mp_mao.HAND_CONNECTIONS)

        for id, cord in enumerate(points.landmark):
            cx, cy = int(cord.x * w), int(cord.y * h)
            cv2.circle(frame, (cx, cy), 4, (255,0,0), -1)
            pontos.append((cx,cy))

    return pontos

In [ ]:
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

while True:
    sucesso, frame = cap.read()
    if not sucesso:
        print("Falha ao capturar imagem")
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resultados_pose = pose.process(frame_rgb)
    resultados_mao = mao.process(frame_rgb)
    
    if resultados_mao.multi_hand_landmarks:
        pontos_mao = processamento_pontos_mao(frame, resultados_mao.multi_hand_landmarks)
    

    if resultados_pose.pose_landmarks:
        pontos_corpo = processamento_pontos_corpo(frame, resultados_pose.pose_landmarks)

    cv2.imshow('Imagem', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
cap.release()
cv2.destroyAllWindows()